# BrainWeb-based multimodal models of 20 normal brains

**Download and Preprocessing for PET-MR Simulations**

This notebook will not re-download/re-process files if they already exist.

- Output data
  + `~/.brainweb/subject_*.npz`: dtype(shape): `float32(127, 344, 344)`
- [Raw data source](http://brainweb.bic.mni.mcgill.ca/brainweb/anatomic_normal_20.html)
  + `~/.brainweb/subject_*.bin.gz`: dtype(shape): `uint16(362, 434, 362)`
- Prerequisites
  + Python: [requirements.txt](requirements.txt) (e.g. `pip install -r requirements.txt`)

----

- Author: Casper da Costa-Luis <<casper.dcl@physics.org>>
- Date: 2017-19
- Licence: [MPLv2.0](https://www.mozilla.org/MPL/2.0)

In [ ]:
from __future__ import print_function, division
%matplotlib notebook
import brainweb
from brainweb import volshow
import numpy as np
from os import path
from tqdm.auto import tqdm
import logging
logging.basicConfig(level=logging.INFO)

## Raw Data

In [ ]:
# download
files = brainweb.get_files()

# read last file
data = brainweb.load_file(files[-1])

# show last subject
print(files[-1])
volshow(data, cmaps=['gist_ncar']);

## Transform

<div style="visibility: hidden">$\ifcsname bm\endcsname\else\newcommand{\bm}[1]{\mathbf{#1}}\fi$</div>
Convert raw image data:

- Siemens Biograph mMR resolution (~2mm) & dimensions (127, 344, 344)
- PET/T1/T2/uMap intensities
- randomised structure for PET/T1/T2
  + $\bm{\theta} \circ (\bm{1} + \gamma[2G_\sigma(\bm{\rho}) - \bm{1}])$
    * $\bm{\rho} = rand(127, 344, 344) \in [0, 1)$
    * Gaussian smoothing $\sigma = 1$
    * $\gamma = \left\{\matrix{1 & \text{for PET}\\ 0.75 & \text{for MR}}\right.$
    * $\bm{\theta}$ is the PET or MR piecewise constant phantom

In [ ]:
brainweb.seed(1337)

for f in tqdm(files, desc="mMR ground truths", unit="subject"):
    vol = brainweb.get_mmr_fromfile(
        f,
        petNoise=1, t1Noise=0.75, t2Noise=0.75,
        petSigma=1, t1Sigma=1, t2Sigma=1)

In [ ]:
# show last subject
print(f)
volshow([vol['PET' ][:, 100:-100, 100:-100],
         vol['uMap'][:, 100:-100, 100:-100],
         vol['T1'  ][:, 100:-100, 100:-100],
         vol['T2'  ][:, 100:-100, 100:-100]],
        cmaps=['hot', 'bone', 'Greys_r', 'Greys_r'],
        titles=["PET", "uMap", "T1", "T2"]);

In [ ]:
# add some lesions
brainweb.seed(1337)
im3d = brainweb.add_lesions(vol['PET'])
volshow(im3d[:, 100:-100, 100:-100], cmaps=['hot']);